In [162]:
# !pip install pymysql
import pandas as pd
import numpy as np
import os
from pathlib import Path
import glob
import csv
import pymysql
from openpyxl import load_workbook
from ipynb.fs.full.Db_Connection import open_conn
pd.set_option('display.max_column', None)
pd.set_option('display.expand_frame_repr', False)

lata_info = pd.read_csv(r'D:/Abbas/Reports/HECIG/Input/Lata_No.csv', dtype='str')
lata_select = lata_info['LATA_NUMBER'].iloc[0]
lata_list = lata_select.split("-")

# dbConnection = sqlEngine.connect()
df_report = pd.read_excel(r'D:/Abbas/Reports/HECIG/Input/Summary.xlsx', dtype=str)
df_report = df_report.replace(np.nan,'')
conn = open_conn()

database = pd.read_sql("""SELECT circuittype AS TYPE, lata_a AS LATA, a_type AS LOC_TYPE, a_unit AS HECIG, a_rrvendor AS VENDOR, a_rrmodel AS MODEL, 
                    a_unitmodel AS PRODUCT, a_rr AS RR 
                    FROM circuitinv WHERE lata_a IN (128,130,132,224,228,236) OR lata_z IN (128,130,132,224,228,236)
                    UNION
                    SELECT circuittype AS TYPE, lata_z AS LATA, z_type AS LOC_TYPE, z_unit AS HECIG, 
                    z_rrvendor AS VENDOR, z_rrmodel AS MODEL, z_unitmodel AS PRODUCT, z_rr AS RR 
                    FROM circuitinv WHERE lata_a IN (128,130,132,224,228,236) OR lata_z IN (128,130,132,224,228,236)""",conn);

allFiles = glob.glob('D:\Abbas\Reports\HECIG\Input\*')
hecig_master_file = [i for i in allFiles if 'Master' in i]
hecig_master = pd.read_excel(hecig_master_file[0], dtype = 'str')
df_output = 'D:/Abbas/Reports/HECIG/Output/'

summary_report = pd.read_excel('D:/Abbas/Reports/Main_Summary.xlsx', dtype=str)
summary_report.columns = summary_report.columns.str.strip()
summary_report.dropna(how='all', inplace=True)
summary_report = summary_report.replace(np.nan,'')

summary_writer = pd.ExcelWriter('D:/Abbas/Reports/HECIG/Output/Summary.xlsx', engine='xlsxwriter')

df_data = database
hg_mstr = hecig_master
hg_mstr['HECIG'] = hg_mstr['HECIG'].str.upper()


for l in range(len(lata_list)):
    
    writer = pd.ExcelWriter(df_output+'L'+lata_list[l]+'_Hecig.xlsx', engine='xlsxwriter')
    
    # Creating columns for further clasifications
    HECIG_LIST = ['DEMARC', 'POT', 'NCTE', 'POI', 'H-POINT', 'TMUX-ADM', 'DSX']
    df_data['HECIG_CLEAN'] = df_data['HECIG'].str.extract(r'([a-zA-Z0-9- ]+)', expand=False).str.strip()
    df_data['FREQ'] = df_data['HECIG'].str.find('_',0,1)
    
    lata_data = df_data.loc[df_data['LATA'] == lata_list[l]]
    
    # Valid Hecig dataframe and filtering
    valid_hg_df = lata_data[lata_data['FREQ'] == -1]
    valid_hg =  valid_hg_df[~valid_hg_df['HECIG_CLEAN'].isin(HECIG_LIST)]
    valid_hg  = valid_hg.loc[(valid_hg['VENDOR']=='') & (valid_hg['MODEL']=='') & (valid_hg['PRODUCT']=='') & (valid_hg['RR']=='')]
    if valid_hg.empty:
        summary_report['L'+lata_list[l]][22] = 'P'
    else:
        summary_report['L'+lata_list[l]][22] = 'F'
        valid_hg.to_excel(writer, sheet_name='Valid_Hecig', index=False)
        
    
    # Invalid Hecig dataframe and filtering
    invalid_hg_df = lata_data[lata_data['FREQ'] != -1]
    invalid_hg =  invalid_hg_df[~invalid_hg_df['HECIG_CLEAN'].isin(HECIG_LIST)]
    invalid_hg  = invalid_hg.loc[(invalid_hg['VENDOR']!='') & (invalid_hg['MODEL']!='') & (invalid_hg['PRODUCT']!='') & (invalid_hg['RR']!='')]
    if invalid_hg.empty:
        summary_report['L'+lata_list[l]][23] = 'P'
    else:
        summary_report['L'+lata_list[l]][23] = 'F'
        invalid_hg.to_excel(writer, sheet_name='Invalid_Hecig', index=False)
    
    # All Demarc type hecig validation
    demr_type_hg = lata_data[lata_data['HECIG_CLEAN'].isin(HECIG_LIST)]
    demr_type_hg = demr_type_hg.loc[(demr_type_hg['VENDOR']!='') & (demr_type_hg['MODEL']!='') & (demr_type_hg['PRODUCT']!='') & (demr_type_hg['RR']!='')]
    if demr_type_hg.empty:
        summary_report['L'+lata_list[l]][24] = 'P'
    else:
        summary_report['L'+lata_list[l]][24] = 'F'
        demr_type_hg.to_excel(writer, sheet_name='DmrcType_Hecig', index=False)
    
    # Not present in the Master file
    not_in_mastr = valid_hg_df[~valid_hg_df['HECIG_CLEAN'].isin(hg_mstr['HECIG'])]
    if not_in_mastr.empty == False:
        not_in_mastr.to_excel(writer, sheet_name='Not_In_Master', index=False)
    
    # Not present in the Maestro app
    not_in_maes = hg_mstr[~hg_mstr['HECIG'].isin(valid_hg_df['HECIG_CLEAN'])]
    if not_in_maes.empty == False:
        not_in_maes.to_excel(writer, sheet_name='Not_In_Maes', index=False)
        
    summary_report.to_excel(summary_writer, sheet_name='Summary', index=False)
    
    writer.save()
summary_writer.save()